# NBA.com stats crawler
Using selenium and BeautifulSoup with Python 3.x
> LATIA (2023, Fall) TA: [王修佑](https://github.com/whyhugo)

In [1]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from lxml import etree
import time

In [2]:
'''不想彈出視窗可以用下面這段 code
chrome_options = Options()
chrome_options.add_argument('--headless')
, options=chrome_options
'''

url = 'https://www.nba.com/stats/players/traditional'
driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')
html = driver.get(url)
sp = soup(driver.page_source, 'lxml')

C:\Users\user\AppData\Local\Temp\ipykernel_25012\1062334439.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


### Get Column Name Using find & find_all

In [3]:
def get_column_name():
    data_column_name = []
    sp = soup(driver.page_source, 'html.parser')
    th_tags = sp.find('tr', class_='Crom_headers__mzI_m').find_all('th')
    for th_tag in th_tags:
        data_column_name.append(th_tag.text)
    data_column_name = data_column_name[1:30]
    return data_column_name

### Click Next Page Button Using XPath

In [4]:
# click next page button
def click_next_page_button():
    time.sleep(3)
    element = driver.find_element('xpath', '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[5]/button[2]')
    driver.execute_script("arguments[0].click();", element)
    time.sleep(2)

### Get Row Data in the Table Using find, find_all & Nested Loop

In [5]:
def get_data():
    data = []
    row = []
    count = 0
    sp = soup(driver.page_source, 'html.parser')
    tr_tags = sp.find('tbody', class_='Crom_body__UYOcU').find_all('tr')
    for tr_tag in tr_tags:
        td_tags = tr_tag.find_all('td')
        for td_tag in td_tags:
            row.append(td_tag.text)
            count += 1
            if count % 30 == 0:
                data.append(row[1:])
                row = []
    return data

In [6]:
try:
    df_list = []
    sp = soup(driver.page_source, 'html.parser')
    total_row = sp.find('div', class_='Pagination_content__f2at7 Crom_cromSetting__Tqtiq').find('div').text[:-5]
    while True:
        print('New Page!')
        time.sleep(2)
        df_list.extend(get_data())
        if len(df_list) == int(total_row):
            print('This is the last page ( •̀ ω •́ )✧')
            break
        else:
            click_next_page_button()
except:
    print('Something went wrong. QAQ')

New Page!
New Page!
New Page!
New Page!
New Page!
New Page!
New Page!
New Page!
This is the last page ( •̀ ω •́ )✧


In [7]:
df = pd.DataFrame(df_list, columns=get_column_name())
df

,Player,Team,Age,GP,W,L,Min,PTS,FGM,FGA,...,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-
0,Luka Doncic,DAL,24,2,2,0,35.1,41.0,14.5,25.0,...,11.5,8.5,2.0,1.0,0.0,1.0,68.6,2.0,1.0,6.5
1,Donovan Mitchell,CLE,27,2,1,1,38.9,35.0,13.0,24.0,...,6.5,5.5,2.5,3.5,1.5,1.0,63.6,0.0,0.0,7.0
2,Cam Thomas,BKN,22,2,0,2,29.2,33.0,12.5,20.0,...,5.0,2.0,0.5,1.0,0.5,1.0,46.0,0.0,0.0,4.0
3,Devin Booker,PHX,26,1,1,0,37.0,32.0,13.0,21.0,...,6.0,8.0,6.0,1.0,0.0,4.0,48.2,0.0,0.0,5.0
4,De'Aaron Fox,SAC,25,3,2,1,35.5,31.3,11.3,23.3,...,4.3,6.0,2.7,1.3,0.7,4.0,48.9,0.0,0.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,Stanley Umude,DET,24,1,1,0,0.7,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0
390,Terry Taylor,CHI,24,1,0,1,1.2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
391,Trayce Jackson-Davis,GSW,23,1,1,0,4.7,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.2,0.0,0.0,7.0
392,Trent Forrest,ATL,25,1,1,0,4.7,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.5,0.0,0.0,-7.0


In [8]:
df.to_csv('nba_stats.csv', index=False, encoding='utf-8')

In [ ]:
driver.quit()